In [17]:
#This Link is Important: https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3 
#Link to Tutorial I followed: https://www.youtube.com/watch?v=SSW9LzOJSus&ab_channel=NicholasRenotte 

In [18]:
!pip install tensorflow==2.10.0 opencv-python matplotlib

In [19]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [20]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
#interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

In [21]:
#Ignore, just for testing
interpreter.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [22]:
print(192 in interpreter.get_input_details()[0]['shape']) #For testing only

True


In [23]:
#Ignore, just for testing
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [24]:
#Drawing Keypoints

def draw_keypoints(frame, keypoints, confidence):
    """
    Draws in the locations of the key joints
    """
    y, x, c = frame.shape #y coordinate, x, coordginate, the channel coordinate
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) #the third coordinate in our keypoints is the confidence value, so we multiply by one to maintain that coordinate
    
    for kp in shaped:
        ky, kx, kp_conf = kp #extracting our y, our x, 
        if kp_conf > confidence:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1) #last three parameters are size, color, fill
    

In [25]:
#Dictionary that represents the connections for the function below
# the value is the letter that corresponds to the color of the edge (they are just here, we are not really using them)
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [26]:
#Drawing Edges

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape #y coordinate, x, coordginate, the channel coordinate
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) #the third coordinate in our keypoints is the confidence value, so we multiply by one to maintain that coordinate
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

In [27]:
#Making Detections

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    #Need to reshape image. Movenet Architecture expects an image of dimensions 192x192x3
    img = frame.copy() #makes a copy of the captured frame and saves it to a variable 'img'
    #to resize, we need to encapsulate it inside another array (that's what expand_dims does)
    #the resizing happens with the resize_with_pad function, where we give it the proper dimensions. 
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192) #For Lightning
    #img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256, 256) #For Thunder

    #Image needs to be represented as a float32 tensor, that is what the below line does
    input_image = tf.cast(img, dtype=tf.float32)
    
    
    #Set input and Output (using the TF interpreter)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    #Make Predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image)) #Setting input details equal to input image
    interpreter.invoke() #invoking (making) our prediction
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index']) # getting output details
    print(keypoints_with_scores)
    
    #Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.5)
    draw_keypoints(frame, keypoints_with_scores, 0.5)
    
    #this tiny snippet is just for naming the camera properly based on the model we r using lol
    if 192 in interpreter.get_input_details()[0]['shape']:
        cv2.imshow('MoveNet Lightning', frame)
    if 256 in interpreter.get_input_details()[0]['shape']:
        cv2.imshow('MoveNet Thunder', frame)
    #Exiting the Camera (Press q)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[[[[0.3271464  0.4718657  0.71926904]
   [0.29388416 0.52033126 0.5604815 ]
   [0.2764214  0.42552075 0.66343844]
   [0.34220418 0.5583868  0.5260926 ]
   [0.31067565 0.34819993 0.554428  ]
   [0.6270484  0.63887453 0.8193223 ]
   [0.52423406 0.17415659 0.7970908 ]
   [0.966038   0.9549861  0.08349246]
   [0.8415496  0.02023799 0.23564331]
   [0.9678454  0.9477017  0.06367252]
   [0.6121887  0.03959467 0.04618006]
   [0.99758494 0.50848573 0.03274653]
   [0.9027369  0.0288186  0.06474754]
   [0.64823127 0.69915235 0.03614818]
   [0.66322744 0.4021536  0.02487086]
   [0.34634334 0.6025924  0.01547406]
   [0.60998714 0.07581349 0.01513407]]]]
[[[[0.3102159  0.4622339  0.62637484]
   [0.28112015 0.50430197 0.6983636 ]
   [0.2585897  0.41126883 0.6770572 ]
   [0.33206066 0.54181087 0.34206742]
   [0.28830284 0.31642422 0.48371086]
   [0.6291745  0.6098215  0.7377298 ]
   [0.5431901  0.13593107 0.5878191 ]
   [0.867691   0.7713343  0.06096436]
   [0.8675066  0.01889231 0.15153836]
   [0.969

[[[[0.29073843 0.3564747  0.40462598]
   [0.26439956 0.40554354 0.6706542 ]
   [0.23938672 0.3071701  0.5656819 ]
   [0.3282246  0.44253552 0.45492527]
   [0.26656228 0.2126182  0.4960606 ]
   [0.6058513  0.5203418  0.72381765]
   [0.5063389  0.05198143 0.35101822]
   [0.87023824 0.67467797 0.10893191]
   [0.4437287  0.00187715 0.04602929]
   [0.41600582 0.53976595 0.01957892]
   [0.40203723 0.03521793 0.20144369]
   [0.99363464 0.45748755 0.04158575]
   [0.90559113 0.02713469 0.05222939]
   [0.6220693  0.57946724 0.06633915]
   [0.53427374 0.12776706 0.014875  ]
   [0.33411324 0.45802733 0.0332733 ]
   [0.40716252 0.01802274 0.03806881]]]]
[[[[0.29073843 0.3564747  0.40462598]
   [0.26439956 0.40554354 0.6706542 ]
   [0.23938672 0.3071701  0.5656819 ]
   [0.3282246  0.44253552 0.45492527]
   [0.26656228 0.2126182  0.4960606 ]
   [0.6058513  0.5203418  0.72381765]
   [0.5063389  0.05198143 0.35101822]
   [0.87023824 0.67467797 0.10893191]
   [0.4437287  0.00187715 0.04602929]
   [0.416

[[[[0.2393236  0.5483304  0.2063563 ]
   [0.22321162 0.62239254 0.19224492]
   [0.1937403  0.52813256 0.37157193]
   [0.27997392 0.68529    0.21758004]
   [0.21113646 0.52278227 0.24404095]
   [0.6154332  0.7041854  0.40248868]
   [0.40821263 0.31030428 0.43243843]
   [0.87532234 0.97267956 0.11762513]
   [0.6791754  0.04168996 0.20749344]
   [0.39687032 0.7225963  0.01912159]
   [0.22693567 0.5379646  0.05629182]
   [0.66372836 0.42187604 0.00576942]
   [0.59510255 0.15321437 0.03494515]
   [0.63292706 0.7092432  0.08567767]
   [0.47214115 0.16949241 0.02764849]
   [0.3645112  0.14731076 0.07037032]
   [0.45683143 0.13223776 0.0436757 ]]]]
[[[[0.29900414 0.6907371  0.22976312]
   [0.25616398 0.73054206 0.22586761]
   [0.24350874 0.6615431  0.16010863]
   [0.29770884 0.7195937  0.06740682]
   [0.24206166 0.5749952  0.17536062]
   [0.52104723 0.63769186 0.14584592]
   [0.41704127 0.4089051  0.25836197]
   [0.8641168  0.94390696 0.11195518]
   [0.64192367 0.1284177  0.10738054]
   [0.666

[[[[0.201592   0.6977931  0.02852405]
   [0.13698986 0.70404667 0.02351624]
   [0.527385   0.8396679  0.23005177]
   [0.11203557 0.7167758  0.02119504]
   [0.11121778 0.6105535  0.01891676]
   [0.22488582 0.694806   0.06442975]
   [0.24060166 0.52591115 0.04745344]
   [0.5209875  0.8186068  0.04290629]
   [0.632657   0.663942   0.07695405]
   [0.60626    0.84625524 0.09610135]
   [0.68350863 0.7935571  0.14111453]
   [0.7441868  0.53268373 0.06112048]
   [0.7663807  0.3650317  0.05903209]
   [0.8316376  0.59989595 0.06727657]
   [0.82853055 0.7631995  0.20892286]
   [0.81327915 0.60896224 0.03385438]
   [0.86831445 0.36831427 0.01568148]]]]
[[[[0.201592   0.6977931  0.02852405]
   [0.13698986 0.70404667 0.02351624]
   [0.527385   0.8396679  0.23005177]
   [0.11203557 0.7167758  0.02119504]
   [0.11121778 0.6105535  0.01891676]
   [0.22488582 0.694806   0.06442975]
   [0.24060166 0.52591115 0.04745344]
   [0.5209875  0.8186068  0.04290629]
   [0.632657   0.663942   0.07695405]
   [0.606

[[[[ 0.0039969   0.59115285  0.01627889]
   [-0.0036351   0.63135517  0.0124418 ]
   [-0.00785324  0.554426    0.01011772]
   [ 0.01453467  0.67358726  0.04882208]
   [ 0.01137823  0.47610766  0.0509803 ]
   [ 0.14625761  0.7689079   0.74001133]
   [ 0.17200969  0.38461918  0.8833365 ]
   [ 0.40283823  0.8261949   0.50458616]
   [ 0.46738386  0.3242311   0.7203349 ]
   [ 0.6437907   0.7571393   0.49899727]
   [ 0.736113    0.3321917   0.68720436]
   [ 0.6431322   0.7227037   0.5266262 ]
   [ 0.6455203   0.4792183   0.69205654]
   [ 0.88386905  0.83780444  0.05509594]
   [ 1.0088975   0.46433616  0.01244534]
   [ 0.6848943   0.7550943   0.04313792]
   [ 0.77754116  0.34401     0.04139987]]]]
[[[[ 0.0039969   0.59115285  0.01627889]
   [-0.0036351   0.63135517  0.0124418 ]
   [-0.00785324  0.554426    0.01011772]
   [ 0.01453467  0.67358726  0.04882208]
   [ 0.01137823  0.47610766  0.0509803 ]
   [ 0.14625761  0.7689079   0.74001133]
   [ 0.17200969  0.38461918  0.8833365 ]
   [ 0.402838

...

[[[[0.47491318 0.7139226  0.61855614]
   [0.4573793  0.7473425  0.35367984]
   [0.44731593 0.6602207  0.7187842 ]
   [0.53923523 0.77134556 0.54459506]
   [0.5192641  0.5747187  0.6480689 ]
   [0.75731605 0.87045133 0.76962805]
   [0.7522845  0.42490828 0.8199793 ]
   [0.85025686 0.9537954  0.01074694]
   [0.8659825  0.35244414 0.03017022]
   [0.5863831  0.8658911  0.05459981]
   [0.73324656 0.47296315 0.00746785]
   [1.0213677  0.8477142  0.00976796]
   [0.39598963 0.5227693  0.0226388 ]
   [0.69397616 0.9563474  0.00701862]
   [0.8105129  0.7192706  0.02376422]
   [0.53072333 0.8599658  0.0611315 ]
   [0.51565844 0.7890862  0.00737406]]]]
[[[[0.47573018 0.71846366 0.6773336 ]
   [0.45685562 0.751266   0.36537686]
   [0.44900227 0.6628383  0.64118505]
   [0.5385457  0.77206063 0.56180644]
   [0.52027553 0.5749855  0.65692663]
   [0.75767446 0.8707265  0.77937955]
   [0.75196874 0.42116892 0.78739357]
   [0.9576137  0.97061557 0.04846008]
   [0.8657727  0.35215494 0.02845125]
   [0.582

[[[[0.48322368 0.7010639  0.6690807 ]
   [0.4618406  0.7356245  0.5483726 ]
   [0.4516816  0.64601314 0.4231706 ]
   [0.53961354 0.7666387  0.43626395]
   [0.5232465  0.5658022  0.550543  ]
   [0.7567121  0.86719894 0.73797274]
   [0.7480601  0.4221983  0.79922485]
   [0.9575387  0.97086704 0.04993491]
   [0.8640503  0.349941   0.03025927]
   [0.58779895 0.868495   0.10172686]
   [0.5383015  0.51246464 0.00647465]
   [0.45735174 0.84117883 0.01821805]
   [0.3948998  0.52441067 0.02531804]
   [0.7913871  0.9061546  0.02398978]
   [0.8183975  0.71249676 0.02309611]
   [0.536186   0.86577463 0.05761578]
   [0.51339674 0.7919921  0.01061707]]]]
[[[[0.4928609  0.6761694  0.756065  ]
   [0.46631104 0.71401423 0.70236886]
   [0.4620683  0.6243809  0.5121485 ]
   [0.53811383 0.759907   0.5367073 ]
   [0.5320592  0.5522271  0.7383523 ]
   [0.7552129  0.8702948  0.7891629 ]
   [0.7469827  0.4233998  0.7576611 ]
   [0.95633304 0.9711     0.04907015]
   [0.8668939  0.34382546 0.0269648 ]
   [0.578

[[[[0.5094616  0.6613709  0.750385  ]
   [0.47263002 0.70036757 0.69168997]
   [0.4705903  0.6066896  0.70562863]
   [0.52424824 0.7488185  0.57390445]
   [0.5250416  0.5353452  0.74391806]
   [0.75443447 0.8649742  0.7304909 ]
   [0.75701725 0.39923677 0.81492937]
   [0.85928595 0.9522792  0.01203139]
   [0.8670046  0.3382851  0.02010638]
   [0.59635806 0.865761   0.04605895]
   [0.7284958  0.43400073 0.00517579]
   [1.0183034  0.8477224  0.01058256]
   [0.3994088  0.50208545 0.01498988]
   [0.7853278  0.8999278  0.01867912]
   [0.7682225  0.39192793 0.01489228]
   [0.5873982  0.980183   0.01395849]
   [0.4012226  0.28913736 0.01047821]]]]
[[[[0.5094616  0.6613709  0.750385  ]
   [0.47263002 0.70036757 0.69168997]
   [0.4705903  0.6066896  0.70562863]
   [0.52424824 0.7488185  0.57390445]
   [0.5250416  0.5353452  0.74391806]
   [0.75443447 0.8649742  0.7304909 ]
   [0.75701725 0.39923677 0.81492937]
   [0.85928595 0.9522792  0.01203139]
   [0.8670046  0.3382851  0.02010638]
   [0.596

[[[[0.53052187 0.66311646 0.7444391 ]
   [0.49177563 0.7078066  0.5535882 ]
   [0.49246272 0.6100384  0.73516476]
   [0.5486441  0.7618524  0.6730045 ]
   [0.5483246  0.5336573  0.7600984 ]
   [0.7564908  0.8708345  0.8308305 ]
   [0.7582718  0.4075507  0.83583426]
   [0.85724795 0.95530677 0.01453598]
   [0.8699216  0.32457188 0.01317826]
   [0.58296883 0.8587126  0.05321618]
   [0.7437641  0.44899654 0.00510888]
   [0.46837395 0.84728944 0.01756179]
   [0.42312098 0.4835834  0.01769858]
   [0.8041057  0.91557515 0.02622304]
   [0.75106025 0.4099617  0.02006555]
   [0.5404452  0.8400972  0.02369381]
   [0.5343106  0.78098387 0.00900818]]]]
[[[[0.5358838  0.6615178  0.72469914]
   [0.49566108 0.70724934 0.48198584]
   [0.49380502 0.60637164 0.72809255]
   [0.5497302  0.76355577 0.68986666]
   [0.54670274 0.53062284 0.7779701 ]
   [0.7572271  0.8708377  0.83329153]
   [0.7622198  0.39981532 0.7990611 ]
   [0.87089413 0.95717496 0.01847866]
   [0.8862697  0.32062578 0.01408342]
   [0.588

[[[[5.3862220e-01 6.7833626e-01 7.3294616e-01]
   [4.9488375e-01 7.2239959e-01 6.3206041e-01]
   [4.8865694e-01 6.1834311e-01 6.2901729e-01]
   [5.4833543e-01 7.6918352e-01 6.6618460e-01]
   [5.3884959e-01 5.3570193e-01 6.7778647e-01]
   [7.6973987e-01 8.8304985e-01 6.9218200e-01]
   [7.5218344e-01 4.0234229e-01 7.6379037e-01]
   [9.5582676e-01 9.7817242e-01 4.5655064e-02]
   [8.8052744e-01 3.2651466e-01 1.4950049e-02]
   [5.8865488e-01 8.6121219e-01 3.7217595e-02]
   [7.8302038e-01 4.8799857e-01 5.2528181e-03]
   [4.8787123e-01 8.6531603e-01 2.0301057e-02]
   [4.2245960e-01 4.8357618e-01 1.8474193e-02]
   [7.9595923e-01 9.3423319e-01 1.5821680e-02]
   [7.3715848e-01 4.1315231e-01 1.1306987e-02]
   [6.0239542e-01 1.0033383e+00 9.1378465e-03]
   [7.0743459e-01 6.3470823e-01 7.1342342e-04]]]]
[[[[0.5280204  0.6871048  0.5731384 ]
   [0.48713738 0.7297915  0.6465696 ]
   [0.48119462 0.62786865 0.61886775]
   [0.5402607  0.7710494  0.5341327 ]
   [0.535702   0.53920174 0.6441492 ]
   [0.76

In [28]:
keypoints_with_scores.shape #For testing (this is as expected from the documentation)

(1, 1, 17, 3)

In [29]:
# Outputs
# A float32 tensor of shape [1, 1, 17, 3].
        # The first two channels of the last dimension represents the yx coordinates (normalized to image frame, i.e. range in [0.0, 1.0]) of the 17 keypoints 
        #(in the order of: [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]).
        # The third channel of the last dimension represents the prediction confidence scores of each keypoint, also in the range [0.0, 1.0].

In [30]:
#Using the order of body parts given in the above cell
right_eye = keypoints_with_scores[0][0][2]
left_elbow = keypoints_with_scores[0][0][7]
left_elbow #printing one of them out to see what it looks like

array([0.88844776, 0.96147466, 0.02412108], dtype=float32)

In [31]:
np.array(left_elbow[:2]*[480,640]).astype(int) #standardized coordinates to frame coordinates

array([426, 615])

In [32]:
frame.shape

(480, 640, 3)